In [1]:
#Загружаю нужные библиотеки

import pandas as pd
import math

# для юзер-агента
from fake_useragent import UserAgent

# прогресс-бар
from tqdm.notebook import trange, tqdm

# для пауз при парсинге
from time import sleep

# селениум
from selenium import webdriver

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# настройки юзер-агента

ua = UserAgent()

# юзер-агент для Хрома
user_agent = ua.chrome

In [3]:
test_url = 'https://intoli.com/blog/not-possible-to-block-chrome-headless/chrome-headless-test.html'

In [4]:
# настройки веб-драйвера, чтобы скрыть автоматическое управление

opt = webdriver.ChromeOptions()
opt.add_argument(f'user-agent={user_agent}')
opt.add_argument('--disable-blink-features')
opt.add_argument('--disable-blink-features=AutomationControlled')
opt.add_argument('--proxy-server=212.192.198.116:57033') # "девственный" прокси(:
opt.add_experimental_option("excludeSwitches", ["enable-automation"]) # отключает строку про удаленное управление
opt.add_experimental_option('useAutomationExtension', False)

In [5]:
# запускаю драйвер, проверяю настройки

driver = webdriver.Chrome(options=opt)
driver.get(test_url)

In [6]:
url_part_1 = 'https://stom-rating.com/crm/?page='

In [121]:
data = [] # список для данных

for i in tqdm(range(1, 4)): # цикл для перехода по страницам
    
    driver.get(url_part_1 + str(i))
    sleep(2)
    
    crm_blocks = driver.find_elements_by_class_name('crm-ajax-hide') # забираю блоки со страниц
    
    for crm in crm_blocks: # вытаскиваю данные из каждого блока (строки таблицы)
        
        try:
            # название crm
            crm_name = (crm.find_element_by_class_name('td.td_1')
                        .get_attribute('textContent').strip())

            # страница crm на сайте
            crm_page = (crm.find_element_by_class_name('td.td_1')
                        .find_element_by_tag_name('a').get_attribute('href'))


            # рейтинг (от 1 до 5)
            score = (crm.find_element_by_class_name('rating-block')
                      .find_element_by_tag_name('input').get_attribute('value')
                     )

            # стоимость
            price = crm.find_element_by_class_name('td.td_2').text

            # версии продукта - облако / коробка
            product_type = []
            for item in crm.find_element_by_class_name('td.td_3').find_elements_by_css_selector('span.yes'):
                product_type += [item.text]


            # количество внешних сервисов
            external_services = crm.find_element_by_class_name('td.td_4').text

            # помощь по внедрению CRM
            support_type = []

            for item in crm.find_element_by_class_name('td.td_5').find_elements_by_css_selector('span.yes'):
                support_type += [item.text]


            # сайт CRM

            # кликаю по кнопке "На сайт"
            crm.find_element_by_css_selector('input.btn-default-2').click()

            # переключаюсь вкладку
            driver.switch_to.window(driver.window_handles[1])
            sleep(2)

            # получаю ссылку
            web_site = driver.current_url

            # закрываю вкладку
            driver.close()

            # переключаюсь обратно
            driver.switch_to.window(driver.window_handles[0])
            sleep(2)


            data.append([crm_name,
                         web_site,
                         score,
                         price,
                         product_type,
                         external_services,
                         crm_page])
        
        except:
            break

  0%|          | 0/3 [00:00<?, ?it/s]

In [122]:
# Записываю в датафрейм

head_list = ['crm_name',
             'web_site',
             'score',
             'price',
             'product_type',
             'external_services',
             'crm_page']

work = pd.DataFrame(data, columns=head_list)

In [123]:
work.shape

(26, 7)

In [124]:
work.to_excel('C:/Data/Стоматологии/stom_crm.xlsx', index=False)

In [80]:
driver.close()